In [ ]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.dates as md
import datetime as dt
import matplotlib

In [ ]:
matplotlib.rcParams.update({'font.size': 14})

datadir = "/icarus/data/users/gputnam/DMCP2023G/normdata/"


dosave = True
savedir = "./plots_2_1_24/"

In [ ]:
beamdf = pd.concat([pd.read_hdf(datadir + "run1_beam.df"), pd.read_hdf(datadir + "run2_beam.df")])

In [ ]:
evtdf = pd.read_csv(datadir + "Run1Run2_OnBeam.csv")

In [ ]:
grl = np.loadtxt(datadir + "Run1Run2_grl.txt")

In [ ]:
evtdf["goodrun"] = False
for r in grl:
    evtdf.loc[evtdf.run == r, "goodrun"] = True

In [ ]:
dq_cuts = [9642, 9723, 9941]

In [ ]:
# dq_cuts = [8518, 9642, 9723, 9781, 9837, 9851, 9892, 9941]

In [ ]:
evtdf["qualityrun"] = evtdf.goodrun
for r in dq_cuts:
    evtdf.loc[evtdf.run == r, "qualityrun"] = False

In [ ]:
grl

In [ ]:
pot_cut = (beamdf.pot > 2)
hc_cut = (beamdf.horn_current > -202) & (beamdf.horn_current <= -196.4)
bp_cut = (np.abs(beamdf.extrap_hptgt) < 1) & (np.abs(beamdf.extrap_vptgt) < 1)
bw_cut = (beamdf.beam_width_h > 0.57) & (beamdf.beam_width_h <= 1.88) &\
         (beamdf.beam_width_v > 0.57) & (beamdf.beam_width_v <= 1.88)

bq_cut = pot_cut & hc_cut & bp_cut & bw_cut

In [ ]:
times = np.array(list(map(dt.datetime.fromtimestamp, beamdf.time)))

In [ ]:
evttimes = np.array(list(map(dt.datetime.fromtimestamp, evtdf.trigger_timestamp_s)))

In [ ]:
plt.plot(times, beamdf.pot.cumsum()/1e8, label="NuMI Beam")
plt.plot(times[bq_cut], beamdf[bq_cut].pot.cumsum()/1e8, label="Quality Beam")

plt.plot(evttimes, evtdf.totpot_corr.cumsum()/1e8, label="Collected with Valid Trigger")
plt.plot(evttimes[evtdf.goodrun], evtdf[evtdf.goodrun].totpot_corr.cumsum()/1e8, label="Collected in Good Run")
plt.plot(evttimes[evtdf.qualityrun], evtdf[evtdf.qualityrun].totpot_corr.cumsum()/1e8, label="Run Satisfies Data Quality")

plt.legend()
plt.xlabel("Date")
plt.ylabel(r"Protons on Target [$\times 10^{20}$]")
plt.xticks(rotation=30, ha='right')

plt.tight_layout()

if dosave:
    plt.savefig(savedir + "uptime.pdf")
    plt.savefig(savedir + "uptime.svg")

In [ ]:
evtdf[evtdf.qualityrun].totpot_corr.sum()/1e8

In [ ]:
allruns = evtdf.run.unique()
pot_per_run = [evtdf.totpot_corr[evtdf.run == r].sum()/1e8 for r in allruns]

In [ ]:
max_lost_pot = sorted([(p,r) for p,r in zip(pot_per_run, allruns) if r in grl and r in dq_cuts], key=lambda t: t[0], reverse=True)

In [ ]:
max_lost_pot

In [ ]:
evtdf[evtdf.run == 9944].totpot_corr.sum()